In [ ]:

import requests
from aind_data_transfer_service.models.core import (
    UploadJobConfigsV2,
    SubmitJobRequestV2,
    Task,
)

from aind_data_schema_models.modalities import Modality

from datetime import datetime
from dateutil import parser
import shutil
import json
import os

: 

## Upload specific sessions

In [ ]:
def upload_single_session_ephys(session_data, root_folder, ephys_folder, file_name):
    
    # The vr_foraging job_type contains presets for compression and Code Ocean pipelines.
    # Please see http://aind-data-transfer-service/job_params for list of job_types.
    job_type = "ecephys"
    subject_id = session_data['subject_id']

    try:
        acq_datetime = datetime.fromisoformat(session_data['session_start_time'])
    except KeyError:
        acq_datetime = datetime.fromisoformat(session_data['acquisition_start_time'])

    project_name = "Ephys Platform"
    s3_bucket = "default"
    behavior_folder = (
        "/allen/aind/scratch/vr-foraging/data/"+subject_id+"/"+file_name
    )

    metadata_dir = ephys_folder + "/metadata"

    gather_preliminary_metadata = Task(
        job_settings={
            "metadata_dir": metadata_dir,
        }
    )

    behavior_videos_source = (
        f"{behavior_folder}/behavior-videos"
    )
    behavior_source = (
        f"{behavior_folder}/behavior"
    )

    behavior_videos_task = Task(
        job_settings={
            "input_source": behavior_videos_source,
            "compression_requested": {"compression_enum": "gamma"},
        }
    )
    behavior_task = Task(
        job_settings={
            "input_source": behavior_source
        }
    )

    ephys_task = Task(
        job_settings={
            "input_source": root_folder,
        }
    )

    modalities = [Modality.BEHAVIOR, Modality.ECEPHYS]

    modality_transformation_settings = {
        "behavior": behavior_task,
        "behavior_videos": behavior_videos_task,
        "ecephys": ephys_task,
    }
        
    upload_job_configs_v2 = UploadJobConfigsV2(
        job_type=job_type,
        project_name=project_name,
        s3_bucket=s3_bucket,
        subject_id=subject_id,
        acq_datetime=acq_datetime,
        modalities=modalities,
        tasks={
            "gather_preliminary_metadata": gather_preliminary_metadata,
            "modality_transformation_settings": modality_transformation_settings,
        },
    )

    # Add more to the list if needed
    upload_jobs = [upload_job_configs_v2]

    # Optional email address and notification types if desired
    submit_request_v2 = SubmitJobRequestV2(
        upload_jobs=upload_jobs
    )
    post_request_content = submit_request_v2.model_dump(mode="json", exclude_none=True)

    # Please use the production endpoint for submitting jobs and the dev endpoint
    # for running tests.
    endpoint = "http://aind-data-transfer-service"
    # endpoint = "http://aind-data-transfer-service-dev"  # For testing

    # Optionally validate the job request before submitting
    validate_job_response = requests.post(
        url=f"{endpoint}/api/v2/validate_json",
        json=post_request_content,
    )
    # print(validate_job_response.status_code)
    # print(validate_job_response.json())

    # Submit the job request
    submit_job_response = requests.post(
        url=f"{endpoint}/api/v2/submit_jobs",
        json=post_request_content,
    )

In [9]:
mouse = '789907'
list_session_requested = retrieve_sessions(mouse)

In [10]:
directory = os.path.join(base_path, mouse)
files = os.listdir(os.path.join(base_path, mouse))

sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(directory, x)), reverse=False)

# All this segment is to find the correct session without having the specific path
i = 0
passed = False
for file_name in sorted_files:
    cleaned = file_name.replace("-", "").rstrip("Z")
    if cleaned not in list_session_requested:
        print(f"File {file_name} not in list_session_requested")

File 789907_2025-03-21T193412Z not in list_session_requested


In [4]:
# file_name = '754574_20241114T092328'
# file_name = '754575_20241007T130518'
# file_name = '754574_2025-03-20T210503Z'
file_name = '789924_2025-07-16T173812Z'
upload_single_session(file_name)

Send request for session:  2025-07-16T173812Z
